In [2]:
import os
import cv2
import random
from pathlib import Path
from sklearn.model_selection import train_test_split
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [6]:
src = Path(r"C:\Users\sweet\Downloads\archive (7)\data")  # if you have one folder with mask/no_mask subdirs
dst = Path("data")

def create_dirs():
    for split in ["train", "val"]:
        for cls in ["mask","no_mask"]:
            (dst/split/cls).mkdir(parents=True, exist_ok=True)

def resize_and_copy(src_dir, dest_dir, size=(224,224)):
    files = list(Path(src_dir).glob("*.*"))
    for f in files:
        img = cv2.imread(str(f))
        if img is None:
            continue
        img = cv2.resize(img, size)
        out_path = dest_dir / f.name
        cv2.imwrite(str(out_path), img)

def main():
    create_dirs()
    # If you already have organized mask/no_mask folders:
    for cls in ["mask","no_mask"]:
        imgs = list((src/cls).glob("*.*"))
        train, val = train_test_split(imgs, test_size=0.15, random_state=42, stratify=None)
        for p in train:
            dst_path = dst/"train"/cls/p.name
            shutil.copy(p, dst_path)
        for p in val:
            dst_path = dst/"val"/cls/p.name
            shutil.copy(p, dst_path)
    print("Prepared data in data/train and data/val")

if __name__ == "__main__":
    main()


ValueError: With n_samples=0, test_size=0.15 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
DATA_DIR = "data"
BATCH = 32
IMG_SIZE = (224,224)
EPOCHS = 8   # increase if you have time




In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    os.path.join(DATA_DIR,"train"),
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary"
)
val_gen = val_datagen.flow_from_directory(
    os.path.join(DATA_DIR,"val"),
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="binary"
)


In [ ]:
base = MobileNetV2(weights="imagenet", include_top=False, input_shape=(IMG_SIZE[0],IMG_SIZE[1],3))
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
out = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base.input, outputs=out)

# freeze base for faster training
for layer in base.layers:
    layer.trainable = False

model.compile(optimizer=Adam(1e-4), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("models/mask_detector.h5", save_best_only=True, monitor="val_accuracy", mode="max"),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
]

history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=callbacks)


In [ ]:
# Optionally unfreeze last layers and fine-tune:
for layer in base.layers[-30:]:
    layer.trainable = True
model.compile(optimizer=Adam(1e-5), loss="binary_crossentropy", metrics=["accuracy"])
model.fit(train_gen, validation_data=val_gen, epochs=3, callbacks=callbacks)

print("Saved model to models/mask_detector.h5")